In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

plt.rcParams['figure.figsize'] = [20, 8]

In [2]:
import time
import psutil
import os

In [3]:
data=pd.read_csv('C:\\Users\\vaish\\OneDrive\\Desktop\\hpe\\data_new.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)
data_b = data.iloc[:, 0:-3:2]

In [4]:
cont=0.26

In [5]:
scaler = MinMaxScaler()
data_df = scaler.fit_transform(data_b)
df = pd.DataFrame(data_df, columns=data_b.columns, index=data.index)

In [6]:
input_dim = 30  # number of features in the input data
encoding_dim = 10  # number of neurons in the bottleneck layer

In [7]:
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

In [8]:
start_wall_time = time.time()
start_cpu_time = time.process_time()
autoencoder.fit(df, df, epochs=100, batch_size=32)

Epoch 1/100
32/32 [==============================] - 0s 1ms/step - loss: 0.0540
Epoch 2/100
32/32 [==============================] - 0s 983us/step - loss: 0.0443
Epoch 3/100
32/32 [==============================] - 0s 827us/step - loss: 0.0356
Epoch 4/100
32/32 [==============================] - 0s 1ms/step - loss: 0.0264
Epoch 5/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0187
Epoch 6/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0134
Epoch 7/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0104
Epoch 8/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0089
Epoch 9/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0081
Epoch 10/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0076
Epoch 11/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 12/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0072
Epoch 13/100
32/32 [=============

In [9]:
reconstructed_data = autoencoder.predict(df)
# get the end time
end_wall_time = time.time()
end_cpu_time = time.process_time()

mse = np.mean(np.power(df - reconstructed_data, 2), axis=1)

In [10]:
envelope = EllipticEnvelope(contamination=cont)  # Set the contamination parameter here
envelope.fit(df)

EllipticEnvelope(contamination=0.26)

In [11]:
anomalies = pd.DataFrame(index=df.index)
anomalies['prediction'] = np.where(envelope.predict(df) == -1, 1, 0)

In [12]:
# get the execution time
wall_time = end_wall_time - start_wall_time
cpu_time = end_cpu_time - start_cpu_time

In [13]:
print(f'Elapsed time: {int(wall_time)} seconds')
print(f'CPU time: {int(cpu_time)} seconds')

Elapsed time: 5 seconds
CPU time: 1 seconds


In [14]:
print(f'Physical Memory usage: {int(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)} MB') 
print(f'Virtual Memory usage:  {int(psutil.Process(os.getpid()).memory_info().vms / 1024 ** 2)} MB')

Physical Memory usage: 299 MB
Virtual Memory usage:  274 MB


In [17]:
#PERFORMANCE PREDICTION
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
true_values = data['Overall anomaly'].values
predicted_values = anomalies['prediction'].values

precision = precision_score(true_values, predicted_values)
f1 = f1_score(true_values, predicted_values)
recall = recall_score(true_values, predicted_values)
accuracy = accuracy_score(true_values, predicted_values)

In [18]:
print("Precision =", precision)
print("F1 Score =", f1)
print("Recall =", recall)
print("Accuracy =", accuracy)


Precision = 0.2786259541984733
F1 Score = 0.2780952380952381
Recall = 0.27756653992395436
Accuracy = 0.6240079365079365
